# Fine-tuning Question Answering Model

This exam will guide you through loading, preprocessing, and fine-tuning a pre-trained model for a question-answering task using a dataset. Follow the steps carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `distilbert-base-cased` for both the model and tokenizer.
- **Dataset**: You will be using the `christti/squad-augmented-v2` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [2]:
from datasets import load_dataset

squad = load_dataset("christti/squad-augmented-v2", split="train[:7500]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/11.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [3]:
squad = squad.train_test_split(test_size=0.2)

## Step 2: Load the Pretrained Tokenizer and Model

Use the model and tokenizer for the question-answering task.

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-cased-distilled-squad")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

## Step 3: Preprocess the Dataset

Define a function to preprocess the dataset by tokenizing both the context and the question. The function will also calculate the start and end positions of the answers. In the tokenizer you might face a problem if you use `truncation=True` so consider using `truncation='only_first'` if needed.

In [5]:
import re
import string

In [6]:
def preprocess_text(text):
    text = text.lower() # LOWER CASE
    text = text.translate(str.maketrans('', '', string.punctuation)) #REMOVING PUNCTUATION
    text = re.sub(r'\d+', '', text) #REMOVING NUMBERS
    text = re.sub(r'\s+', ' ', text).strip() #REMOVING WHITE SPACE
    return text

In [7]:
def preprocess_function(examples):

    questions = [preprocess_text(q.strip()) for q in examples["question"]]
    contexts = [preprocess_text(c) for c in examples["context"]]

    #Tokenize the questions and contexts
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping") #token positions in the text
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

## Step 4: Define Training Arguments and Initialize the Trainer

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./result",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
)

## Step 5: Fine-tune the Model

Run the training process using the initialized trainer.

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
)
trainer.train()

Step,Training Loss
500,0.601100
1000,0.315200
1500,0.188500
2000,0.099900
2500,0.073000
3000,0.044500
3500,0.030800


TrainOutput(global_step=3750, training_loss=0.18203877080281575, metrics={'train_runtime': 531.7141, 'train_samples_per_second': 112.843, 'train_steps_per_second': 7.053, 'total_flos': 5879389501440000.0, 'train_loss': 0.18203877080281575, 'epoch': 10.0})

## Step 6: Inference

Once the model is trained, perform inference by answering a question based on a context. Use the tokenizer to process the input, and then feed it into the model to get the predicted answer.

In [11]:
model.save_pretrained("./fine-tunned")
tokenizer.save_pretrained("./fine-tunned")

('./fine-tunned/tokenizer_config.json',
 './fine-tunned/special_tokens_map.json',
 './fine-tunned/vocab.txt',
 './fine-tunned/added_tokens.json',
 './fine-tunned/tokenizer.json')

In [12]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model='/content/fine-tunned/', tokenizer='/content/fine-tunned/')
pred = question_answerer(question=question, context=context)
print(pred)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.9999959468841553, 'start': 93, 'end': 95, 'answer': '13'}


In [14]:
print('question: ',question, pred['answer'])

question:  How many programming languages does BLOOM support? 13
